## Preprocessing

In [33]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras 

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# cutoff = 528
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C1570        1
C2190        1
C2600        1
C3700        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1] 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C1234        2
C1267        2
C3200        2
C1256        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [10]:
application_df['CLASSIFICATION'].unique()

array(['C1000', 'C2000', 'C3000', 'C1200', 'C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C2100', 'C1238',
       'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300',
       'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240',
       'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235',
       'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278',
       'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283',
       'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256',
       'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200',
       'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900',
       'C2150'], dtype=object)

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced - cutoff = 1000

classifications_to_replace = ['C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C1238',
       'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300',
       'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240',
       'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235',
       'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278',
       'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283',
       'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256',
       'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200',
       'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900',
       'C2150']

In [12]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
#see which values are categorical
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], prefix='DMMY')

In [15]:
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,DMMY_Other,DMMY_T10,DMMY_T19,DMMY_T3,DMMY_T4,DMMY_T5,DMMY_T6,...,DMMY_1-9999,DMMY_10000-24999,DMMY_100000-499999,DMMY_10M-50M,DMMY_1M-5M,DMMY_25000-99999,DMMY_50M+,DMMY_5M-10M,DMMY_N,DMMY_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'DMMY_Other', 'DMMY_T10',
       'DMMY_T19', 'DMMY_T3', 'DMMY_T4', 'DMMY_T5', 'DMMY_T6', 'DMMY_T7',
       'DMMY_T8', 'DMMY_CompanySponsored', 'DMMY_Family/Parent',
       'DMMY_Independent', 'DMMY_National', 'DMMY_Other', 'DMMY_Regional',
       'DMMY_C1000', 'DMMY_C1200', 'DMMY_C2000', 'DMMY_C2100', 'DMMY_C3000',
       'DMMY_Other', 'DMMY_CommunityServ', 'DMMY_Heathcare', 'DMMY_Other',
       'DMMY_Preservation', 'DMMY_ProductDev', 'DMMY_Association',
       'DMMY_Co-operative', 'DMMY_Corporation', 'DMMY_Trust', 'DMMY_0',
       'DMMY_1-9999', 'DMMY_10000-24999', 'DMMY_100000-499999', 'DMMY_10M-50M',
       'DMMY_1M-5M', 'DMMY_25000-99999', 'DMMY_50M+', 'DMMY_5M-10M', 'DMMY_N',
       'DMMY_Y'],
      dtype='object')

In [17]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df['IS_SUCCESSFUL']
X = application_df[['STATUS', 'ASK_AMT', 'DMMY_Other', 'DMMY_T10',
       'DMMY_T19', 'DMMY_T3', 'DMMY_T4', 'DMMY_T5', 'DMMY_T6', 'DMMY_T7',
       'DMMY_T8', 'DMMY_CompanySponsored', 'DMMY_Family/Parent',
       'DMMY_Independent', 'DMMY_National', 'DMMY_Other', 'DMMY_Regional',
       'DMMY_C1000', 'DMMY_C1200', 'DMMY_C2000', 'DMMY_C2100', 'DMMY_C3000',
       'DMMY_Other', 'DMMY_CommunityServ', 'DMMY_Heathcare', 'DMMY_Other',
       'DMMY_Preservation', 'DMMY_ProductDev', 'DMMY_Association',
       'DMMY_Co-operative', 'DMMY_Corporation', 'DMMY_Trust', 'DMMY_0',
       'DMMY_1-9999', 'DMMY_10000-24999', 'DMMY_100000-499999', 'DMMY_10M-50M',
       'DMMY_1M-5M', 'DMMY_25000-99999', 'DMMY_50M+', 'DMMY_5M-10M', 'DMMY_N',
       'DMMY_Y']]
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [28]:
X_train.shape

(25724, 55)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=55))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                4480      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,941
Trainable params: 6,941
Non-trainable params: 0
_________________________________________________________________


In [35]:
#Create a callback that saves the model's weights every five epochs.
saving_callback = keras.callbacks.ModelCheckpoint('checkpoints/', save_weights_only=True, save_freq=5)
    
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [36]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[saving_callback]) 

Epoch 1/100
804/804 [==============================] - 14s 16ms/step - loss: 0.5357 - accuracy: 0.7398
Epoch 2/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5352 - accuracy: 0.7398
Epoch 3/100
804/804 [==============================] - 14s 17ms/step - loss: 0.5349 - accuracy: 0.7405
Epoch 4/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5349 - accuracy: 0.7399
Epoch 5/100
804/804 [==============================] - 14s 18ms/step - loss: 0.5360 - accuracy: 0.7399
Epoch 6/100
804/804 [==============================] - 14s 17ms/step - loss: 0.5350 - accuracy: 0.7404
Epoch 7/100
804/804 [==============================] - 14s 17ms/step - loss: 0.5347 - accuracy: 0.7407
Epoch 8/100
804/804 [==============================] - 14s 17ms/step - loss: 0.5345 - accuracy: 0.7401
Epoch 9/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5346 - accuracy: 0.7401
Epoch 10/100
804/804 [==============================] - 13s 16ms/step - l

804/804 [==============================] - 13s 17ms/step - loss: 0.5322 - accuracy: 0.7417
Epoch 80/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5327 - accuracy: 0.7416
Epoch 81/100
804/804 [==============================] - 14s 17ms/step - loss: 0.5328 - accuracy: 0.7400
Epoch 82/100
804/804 [==============================] - 14s 17ms/step - loss: 0.5328 - accuracy: 0.7405
Epoch 83/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5332 - accuracy: 0.7406
Epoch 84/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5329 - accuracy: 0.7413
Epoch 85/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5325 - accuracy: 0.7409
Epoch 86/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5322 - accuracy: 0.7413
Epoch 87/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5321 - accuracy: 0.7414
Epoch 88/100
804/804 [==============================] - 13s 16ms/step - loss:

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5808 - accuracy: 0.7273 - 1s/epoch - 4ms/step
Loss: 0.5807662606239319, Accuracy: 0.7273469567298889


In [38]:
# Export our model to HDF5 file
nn.save('models/AlphabetSoupCharity.h5')


## Report

Overview of the analysis: Explain the purpose of this analysis.

The purpose of this analysis was to use Tensorflow to design a binary classifier that can predict whether applicants will be successful if funded by Alphabet Soup.  This would be completed using the features in the provided dataset.

Results: Using bulleted lists and images to support your answers, address the following questions.


Data Preprocessing

* Target variable for the model: 'IS_SUCCESSFUL' (whether or not the applicants used money effectively in the past) 
* What variable(s) are the features for your model?
* What variable(s) should be removed from the input data because they are neither targets nor features?

Compiling, Training, and Evaluating the Model

* How many neurons, layers, and activation functions did you select for your neural network model, and why?
* Were you able to achieve the target model performance?
* What steps did you take in your attempts to increase model performance?